Learning Objectives:
   1. Exclude certain keywords from source text before giving responses
   2. Decide similarity threshold score, top k nodes
   3. Count #tokens while generating responses
   2. Build a Metadataextractor

In [1]:
import os

In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os
import numexpr as ne
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [4]:
import openai

In [5]:
import os
# os.environ["OPENAI_API_KEY"] = "<Your API Key"
import openai
openai.api_key = "<Your API Key"

In [6]:
os.getcwd()

'd:\\LLMOps\\Udemy\\LLAMAINDEXSCRIPTS'

 Step 1. setting up a text indexing system using the llama_index library. It involves token counting, creating a service context, loading or creating an index, and persisting the index to disk.

In [7]:
import tiktoken
from llama_index import ServiceContext
from llama_index.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("text-embedding-ada-002").encode,
    verbose=True
)

callback_manager= CallbackManager([token_counter])
service_context = ServiceContext.from_defaults(callback_manager= callback_manager)

In [8]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, ServiceContext 


try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/cache/andrew/sleep')
    
    #Attempts to load an index from the specified storage context using the previously created service_context.
    index = load_index_from_storage(storage_context,service_context=service_context)
    print('loading from disk')
except:
    print("except block encountered")
    documents = SimpleDirectoryReader('./assets/AndrewHuberman/sleep').load_data() #chunking up data, creating nodes from docx; All these nodes and 
    #documents need to be embedde in to vectors and Llama-Index needs to setup in-memory vector store and create an Index.
    # service_context = ServiceContext.from_defaults("local:/BAAI/bge-base-en-v1.5")
    
    
    # This involves embedding the documents into vectors and setting up an in-memory vector store.
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    index.storage_context.persist(persist_dir='./storage/cache/andrew/sleep/')
    print('persisting to disk')



INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [9]:
index

In [10]:
openai.log = "debug"

In [11]:
from llama_index.prompts import PromptTemplate

text_qa_template_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Using both the context information and also using your own knowledge, "
    "answer the question: {query_str}\n"
    "If the context isn't helpful, you can also answer the question on your own.\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [12]:
text_qa_template_str = (
    "You are an Andrew huberman assistant that can read Andrew Huberman podcast notes.\n"
    "Always answer the query only using the provided context information, "
    "and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines."
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [13]:
#Once you've an Index, you can use it as a query engine.
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How does sleep enhance learning memory?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2058
LLM Completion Token Usage: 166


In [14]:
print(response)

Sleep enhances learning and memory by facilitating motor learning and the learning of specific details about events. Slow wave sleep, which primarily occurs in the early part of the night, is important for motor skill learning and the acquisition of detailed information. During slow wave sleep, there is a release of neuromodulators such as norepinephrine and serotonin, which contribute to the consolidation of learning. Additionally, REM sleep, which occurs throughout the night with a larger percentage towards morning, is also involved in learning and memory processes. Consistently getting a regular amount of sleep each night is more important for learning and performance than simply getting more sleep overall. Variations in sleep duration can lead to a reduction in performance on exams. Therefore, maintaining a consistent sleep schedule and avoiding variations in sleep duration can optimize learning and memory consolidation during sleep.


In [16]:
from llama_index.response.pprint_utils import pprint_response


In [17]:
#Method 2
query_engine = index.as_query_engine()
response = query_engine.query("What can I do to sleep better")
pprint_response(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2025
LLM Completion Token Usage: 185
Final Response: To sleep better, there are several things you can try.
First, you can lower your core body temperature by taking a hot shower
or bath and then cooling off. This can help bring your nervous system
into a state of calmness. Additionally, you can establish a consistent
sleep routine by following a morning and afternoon routine, avoiding
caffeine in the hours leading up to bedtime, and creating a cool and
dark sleep environment. Elevating your feet or the head side of your
bed can also improve the depth of sleep. If you suffer from sleep
apnea, it is important to ad

In [18]:
pprint_response(response)

Final Response: To sleep better, there are several things you can try.
First, you can lower your core body temperature by taking a hot shower
or bath and then cooling off. This can help bring your nervous system
into a state of calmness. Additionally, you can establish a consistent
sleep routine by following a morning and afternoon routine, avoiding
caffeine in the hours leading up to bedtime, and creating a cool and
dark sleep environment. Elevating your feet or the head side of your
bed can also improve the depth of sleep. If you suffer from sleep
apnea, it is important to address it as it can have detrimental
effects on your health. One way to improve sleep apnea is by training
yourself to be a nose breather during sleep, which can be done by
taping your mouth shut. Finally, there are sleep supplements such as
magnesium threonate, apigenin, and theanine that can enhance the
quality of sleep.


In [21]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='a7f08eb8-31e6-4124-b12e-90a25fae0d81', embedding=None, metadata={'file_path': 'assets\\AndrewHuberman\\sleep\\84_Sleep_Toolkit_Tools_for_Optimizing_Sleep_&_SleepWake_Timing_Huberman_Lab_Podcast_84.txt', 'file_name': '84_Sleep_Toolkit_Tools_for_Optimizing_Sleep_&_SleepWake_Timing_Huberman_Lab_Podcast_84.txt', 'file_type': 'text/plain', 'file_size': 113543, 'creation_date': '2023-12-11', 'last_modified_date': '2023-12-11', 'last_accessed_date': '2023-12-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='assets\\AndrewHuberman\\sleep\\84_Sleep_Toolkit_Tools_for_Optimizing_Sleep_&_SleepWake_Timing_Huberman_Lab_Podcast_84.txt', node_type=<ObjectType.DOCUMENT: '4'>, metadata={

In [22]:
pprint_response(response, show_source=True)

Final Response: To sleep better, there are several things you can try.
First, you can lower your core body temperature by taking a hot shower
or bath and then cooling off. This can help bring your nervous system
into a state of calmness. Additionally, you can establish a consistent
sleep routine by following a morning and afternoon routine, avoiding
caffeine in the hours leading up to bedtime, and creating a cool and
dark sleep environment. Elevating your feet or the head side of your
bed can also improve the depth of sleep. If you suffer from sleep
apnea, it is important to address it as it can have detrimental
effects on your health. One way to improve sleep apnea is by training
yourself to be a nose breather during sleep, which can be done by
taping your mouth shut. Finally, there are sleep supplements such as
magnesium threonate, apigenin, and theanine that can enhance the
quality of sleep.
______________________________________________________________________
Source Node 1/2
Node 

In [23]:
### What if you want to send more context to LLM; If you want to send 4 notes of data to your LLM.

query_engine = index.as_query_engine()


In [24]:
index

In [25]:
from llama_index.retrievers import VectorIndexRetriever #used for retrieving vectors or indexed data.
from llama_index.query_engine import RetrieverQueryEngine #used for handling queries with the retriever.
from llama_index.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor #are likely used for post-processing the results of the query.
from llama_index.response_synthesizers import get_response_synthesizer

retriever = VectorIndexRetriever( index= index, 
                                     similarity_top_k=4,)


s_processor = SimilarityPostprocessor( similarity_cutoff= 0.83)
k_processor = KeywordNodePostprocessor( exclude_keywords= ["supplements"])
response_synthesizer = get_response_synthesizer ( response_mode= "no_text")

query_engine = RetrieverQueryEngine(
                                    retriever=retriever,
                                    node_postprocessors=[k_processor, s_processor],
                                    
                                )
                                

In [27]:

response = query_engine.query("What can I do to sleep better")
pprint_response(response, show_source=True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 3020
LLM Completion Token Usage: 222
Final Response: To sleep better, there are several things you can try.
First, you can experiment with using earplugs and an eye mask to see
if they help or disrupt your sleep. Elevating your feet with a pillow
or by raising the end of your bed can also improve the depth of your
sleep. However, if you suffer from acid reflux, it's better to elevate
the head side of your bed instead. Sleep apnea can be a serious issue,
so if you suspect you have it, it's important to seek medical advice.
Being a nose breather during sleep can help alleviate sleep apnea, and
you can train yourself

In [28]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.response_synthesizers import get_response_synthesizer

retriever = VectorIndexRetriever( index= index, 
                                     similarity_top_k=4,)


s_processor = SimilarityPostprocessor( similarity_cutoff= 0.83)
k_processor = KeywordNodePostprocessor( exclude_keywords= ["supplements"])
response_synthesizer = get_response_synthesizer ( response_mode= "no_text")  # "Compact" is another response mode

query_engine = RetrieverQueryEngine(
                                    retriever=retriever,
                                    node_postprocessors=[k_processor],
                                    response_synthesizer=response_synthesizer
                                    
                                )
                                
                                


In [29]:

response = query_engine.query("What can I do to sleep better")
pprint_response(response, show_source=True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
Final Response:
______________________________________________________________________
Source Node 1/3
Node ID: 6e3daa88-eab7-47f6-8ef8-4b9abbe2c04d
Similarity: 0.8471691941863171
Text: I'm one such person. Although, I have family members that like
using earplugs when they sleep. So it's really up to you. You have to
see whether or not those earplugs help or disrupt your sleep. For me,
they're no good. For some people, they really enjoy them. I don't use
an eye mask unless I'm sleeping in a really bright environment or I
need t...
______________________________________________________________________
Source Node 2/3
Node ID: 5d39fef7-6f9a-4647-a8a4-2115bed76558
Similarity: 0.840511601959647
Text: So if you're somebody that enjoys hot baths, hot showers, or hot
tubs, evening and nighttime is going to be th

In [40]:
########## Count number of tokens while using llama-index

import tiktoken
from llama_index import ServiceContext
from llama_index.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("text-embedding-ada-002").encode,
    verbose=True
)

callback_manager= CallbackManager([token_counter])
service_context = ServiceContext.from_defaults(callback_manager= callback_manager)

from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, ServiceContext 


try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/cache/andrew/sleep')
    index = load_index_from_storage(storage_context,service_context=service_context)
    print('loading from disk')
except:
    print("except block encountered")
    documents = SimpleDirectoryReader('./assets/AndrewHuberman/sleep').load_data() #chunking up data, creating nodes from docx; All these nodes and 
    #documents need to be embedde in to vectors and Llama-Index needs to setup in-memory vector store and create an Index.
    # service_context = ServiceContext.from_defaults("local:/BAAI/bge-base-en-v1.5")
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    index.storage_context.persist(persist_dir='./storage/cache/andrew/sleep/')
    print('persisting to disk')



INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [42]:
#Method 2
query_engine = index.as_query_engine()
response = query_engine.query("What can I do to sleep better")
print("Embedding tokens :", token_counter.total_embedding_token_count,'\n',
      "LLM Prompts :", token_counter.prompt_llm_token_count,'\n',
      "LLM Completitions  :", token_counter.completion_llm_token_count,'\n',
      "Total LLM Token Count  :", token_counter.total_llm_token_count,'\n',
      )
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2025
LLM Completion Token Usage: 185
Embedding tokens : 14 
 LLM Prompts : 4050 
 LLM Completitions  : 391 
 Total LLM Token Count  : 4441 

To sleep better, there are several things you can try. First, you can lower your core body temperature by taking a hot shower or bath and then cooling off. This can help bring your nervous system into a state of calmness. Additionally, you can establish a consistent sleep routine by following a morning and afternoon routine, avoiding caffeine in the hours leading up to bedtime, and creating a cool and dark sleep environment. Elevating your feet or the head side of your bed ca

In [43]:
#Method 2
query_engine = index.as_query_engine()
response = query_engine.query("What can I do to sleep better")
print("Embedding tokens :", token_counter.total_embedding_token_count,'\n',
      "LLM Prompts :", token_counter.prompt_llm_token_count,'\n',
      "LLM Completitions  :", token_counter.completion_llm_token_count,'\n',
      "Total LLM Token Count  :", token_counter.total_llm_token_count,'\n',
      )
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2025
LLM Completion Token Usage: 175
Embedding tokens : 21 
 LLM Prompts : 6075 
 LLM Completitions  : 566 
 Total LLM Token Count  : 6641 

To sleep better, there are several things you can try. First, you can lower your core body temperature by taking a hot shower or bath and then cooling off. This can help bring your nervous system into a state of calmness. Additionally, you can establish a consistent sleep routine by following a morning and afternoon routine, avoiding caffeine in the hours leading up to bedtime, and creating a cool and dark sleep environment. Elevating your feet or the head side of your bed ca

In [44]:
#To reset token_counts

token_counter.reset_counts()

#Method 2
query_engine = index.as_query_engine()
response = query_engine.query("What can I do to sleep better")
print("Embedding tokens :", token_counter.total_embedding_token_count,'\n',
      "LLM Prompts :", token_counter.prompt_llm_token_count,'\n',
      "LLM Completitions  :", token_counter.completion_llm_token_count,'\n',
      "Total LLM Token Count  :", token_counter.total_llm_token_count,'\n',
      )
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2025
LLM Completion Token Usage: 183
Embedding tokens : 7 
 LLM Prompts : 2025 
 LLM Completitions  : 183 
 Total LLM Token Count  : 2208 

To sleep better, there are several things you can try. First, you can lower your core body temperature by taking a hot shower or bath and then cooling off. This can help calm your nervous system and prepare your body for sleep. Additionally, you should avoid consuming caffeine or alcohol, as they can disrupt your sleep patterns. It's also important to create a sleep-friendly environment by dimming the lights, cooling down the room, and using tools like earplugs or an eye mask 

In [46]:
################ Refining prompts with user and system prompts

from llama_index.prompts import PromptTemplate

text_qa_template_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Using both the context information and also using your own knowledge, "
    "answer the question: {query_str}\n"
    "If the context isn't helpful, you can also answer the question on your own.\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

#Once you've an Index, you can use it as a query engine.
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How does sleep enhance learning memory?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2013
LLM Completion Token Usage: 166


In [50]:
text_qa_template_str = (
    "You are an Andrew huberman assistant that can read Andrew Huberman podcast notes.\n"
    "Always answer the query only using the provided context information, "
    "and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines."
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How does sleep enhance learning memory?")

print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2058
LLM Completion Token Usage: 166
Sleep enhances learning and memory by facilitating motor learning and the learning of specific details about events. Slow wave sleep, which primarily occurs in the early part of the night, is important for motor skill learning and the acquisition of detailed information. During slow wave sleep, there is a release of neuromodulators such as norepinephrine and serotonin, which contribute to the consolidation of learning. Additionally, REM sleep, which occurs throughout the night with a larger percentage towards morning, is also involved in learning and memory processes. Consisten

In [51]:
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How does sleep enhance learning memory?")

print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
LLM Prompt Token Usage: 2058
LLM Completion Token Usage: 166
Sleep enhances learning and memory by facilitating motor learning and the learning of specific details about events. Slow wave sleep, which primarily occurs in the early part of the night, is important for motor skill learning and the acquisition of detailed information. During slow wave sleep, there is a release of neuromodulators such as norepinephrine and serotonin, which contribute to the consolidation of learning. Additionally, REM sleep, which occurs throughout the night with a larger percentage towards morning, is also involved in learning and memory processes. Consisten

In [54]:
# from pprint import pprint
print(response.source_nodes)
# pprint(response.source_nodes)

[NodeWithScore(node=TextNode(id_='93e79c9d-7fc4-40ff-a0db-a1597fcd6516', embedding=None, metadata={'file_path': 'assets\\AndrewHuberman\\sleep\\05_Understanding_and_Using_Dreams_to_Learn_and_to_Forget_Huberman_Lab_Podcast_5.txt', 'file_name': '05_Understanding_and_Using_Dreams_to_Learn_and_to_Forget_Huberman_Lab_Podcast_5.txt', 'file_type': 'text/plain', 'file_size': 75844, 'creation_date': '2023-12-11', 'last_modified_date': '2023-12-11', 'last_accessed_date': '2023-12-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='92c4d4c0-d36d-4c26-9bb9-01932cc42c6b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'assets\\AndrewHuberman\\sleep\\05_Understanding_and_Using_Dreams_to_Learn_and_to_Forge

In [18]:
## Documents and Nodes in LlamaIndex: Enhances Chatbot's Contextual understanding
from llama_index import SimpleDirectoryReader
andrew_gina_docs= SimpleDirectoryReader(input_files=["./assets/AndrewHuberman/sleep/115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt"]).load_data()

In [19]:
print(len(andrew_gina_docs))

1


In [22]:
from llama_index.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor,

)

from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=20)



extractors = [
    TitleExtractor(nodes=5),
    QuestionsAnsweredExtractor(questions=3)
]

transformations = [text_splitter] + extractors




In [26]:
# # import asyncio
# # await asyncio.sleep(10, result='hello')

# import nest_asyncio
# nest_asyncio.apply()

In [31]:
from llama_index.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

andrew_gina_nodes = pipeline.run(documents=andrew_gina_docs)

467


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 20%|██        | 1/5 [00:01<00:04,  1.15s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

 97%|█████████▋| 60/62 [01:26<00:03,  1.51s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 98%|█████████▊| 61/62 [01:28<00:01,  1.37s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 62/62 [01:33<00:00,  1.51s/it]


In [32]:
print(len(andrew_gina_nodes))

62


In [33]:
andrew_gina_nodes[2].metadata


{'file_path': 'assets\\AndrewHuberman\\sleep\\115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt',
 'file_name': '115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt',
 'file_type': 'text/plain',
 'file_size': 126154,
 'creation_date': '2023-12-11',
 'last_modified_date': '2023-12-11',
 'last_accessed_date': '2023-12-14',
 'document_title': 'The Comprehensive Guide to Sleep: Understanding its Importance for Mental and Physical Health, Enhancing Quality and Performance, and Exploring Different Phases and Functions.',
 'questions_this_excerpt_can_answer': "1. What are some factors that contribute to a good night's sleep and how can they be optimized?\n2. How does the quality of sleep impact mental and physical health, as well as overall performance?\n3. What are some innovative technologies or products available to improve sleep quality and track sleep patterns?"}

In [34]:
import random
sample_gina_nodes = random.sample(andrew_gina_nodes, 5)
for node in sample_gina_nodes:
    print(node.metadata)

{'file_path': 'assets\\AndrewHuberman\\sleep\\115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt', 'file_name': '115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt', 'file_type': 'text/plain', 'file_size': 126154, 'creation_date': '2023-12-11', 'last_modified_date': '2023-12-11', 'last_accessed_date': '2023-12-14', 'document_title': 'The Comprehensive Guide to Sleep: Understanding its Importance for Mental and Physical Health, Enhancing Quality and Performance, and Exploring Different Phases and Functions.', 'questions_this_excerpt_can_answer': '1. How long did participants in the study sleep when they were placed in a semi-darkened room with only a bed for 12 hours every day for a month?\n2. Can oversleeping make us groggy the next day?\n3. What happens to cognitive function when individuals consistently deprive themselves of sleep and only sleep for four to four and a half hours a night?'}
{'fil

In [46]:
# from llama_index import ServiceContext, VectorStoreIndex

# service_context = ServiceContext.from_defaults(node_parser=andrew_gina_nodes)
# index = VectorStoreIndex.from_documents(andrew_gina_docs, service_context=service_context)


# index_no_metadata = VectorStoreIndex(
#     nodes=andrew_gina_nodes,
#     service_context=ServiceContext.from_defaults(),
# )

# response = index.as_query_engine().query("How does consistent bedtime contribute to good neurological health as we age?")


In [47]:
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL


service_context = ServiceContext.from_defaults(
    text_splitter=text_splitter
)
question_gen = LLMQuestionGenerator.from_defaults(
    service_context=service_context,
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

In [49]:
from llama_index import VectorStoreIndex
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

final_engine_no_metadata = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine_no_metadata,
            metadata=ToolMetadata(
                name="sec_filing_documents",
                description="financial information on companies",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=True,
)

TypeError: 'LLMQuestionGenerator' object is not iterable